In [36]:
import os
import copy
import json
import numpy as np
import plotly.graph_objects as go

from pymoo.indicators.hv import Hypervolume, HV
from heuristics.mip import mipPy
from movements.allocate import allocate
from classes.objectives import Objectives
from utils.instance import parse_data, read_instance
from utils.pareto import dominates
from utils.population import generate_first_population
from utils.verifier import remove_objectives_duplicates, verifier
from utils.dataManipulation import allocate_professors, allocate_reservations, create_variable_classrooms, create_variable_meetings, create_variable_professors, find_preferences, find_relatives_meetings

In [12]:
def normalize_values(values_1, values_2, values_3):
    max_value = max(max(values_1), max(values_2), max(values_3))
    min_value = min(min(values_1), min(values_2), min(values_3))
    return [((value - min_value) / (max_value - min_value)) if (max_value - min_value != 0) else 1 for value in values_1],\
           [((value - min_value) / (max_value - min_value)) if (max_value - min_value != 0) else 1 for value in values_2],\
           [((value - min_value) / (max_value - min_value)) if (max_value - min_value != 0) else 1 for value in values_3]

def nondominated_sort(solutions):
    fronts = []
    domination_count = [0] * len(solutions)
    dominated_solutions = [[] for _ in range(len(solutions))]

    for i in range(len(solutions)):
        for j in range(i + 1, len(solutions)):
            if dominates(solutions[i], solutions[j]):
                dominated_solutions[i].append(j)
                domination_count[j] += 1
            elif dominates(solutions[j], solutions[i]):
                dominated_solutions[j].append(i)
                domination_count[i] += 1

    front = []
    for i in range(len(solutions)):
        if domination_count[i] == 0:
            front.append(i)

    while front:
        next_front = []
        for i in front:
            for j in dominated_solutions[i]:
                domination_count[j] -= 1
                if domination_count[j] == 0:
                    next_front.append(j)
        fronts.append(front)
        front = next_front

    return fronts

In [6]:
objectives = Objectives()

# Reading instance data
filename = 'input-seed-6-size-1000.json'
instance_data = read_instance(filename)

# Creating variables
instance = parse_data(instance_data)
classrooms = create_variable_classrooms(instance)
professors = create_variable_professors(instance)
meetings = create_variable_meetings(instance, objectives)

# Allocating professors, reservations, preferences and looking for relatives meetings
relatives_meetings = find_relatives_meetings(meetings)
allocate_professors(meetings, professors)
allocate_reservations(classrooms, instance["reservations"])
find_preferences(meetings, instance["preferences"])

# Saving original solution
print(f"[INFO] Saving original solution")
original_meetings = copy.deepcopy(meetings)
original_classrooms = copy.deepcopy(classrooms)
original_objectives = copy.deepcopy(objectives)
original_solution = {
    "meetings": original_meetings,
    "classrooms": original_classrooms,
    "objectives": original_objectives
}
verifier(original_solution)

print(len(original_meetings))
print(len(classrooms))
print(len(instance_data['schedules']))

[INFO] Reading instance file 'input-seed-6-size-1000.json'
[INFO] Parsing instance data
[INFO] Parsing schedules
[INFO] Parsing buildings
[INFO] Parsing classrooms
[INFO] Parsing professors
[INFO] Parsing subjects
[INFO] Parsing meetings
[INFO] Parsing preferences
[INFO] Parsing restrictions
[INFO] Parsing reservations

[INFO] Creating variable classrooms


100%|██████████| 36/36 [00:00<00:00, 36080.03it/s]



[INFO] Creating variable professors


  0%|          | 0/226 [00:00<?, ?it/s]

100%|██████████| 226/226 [00:00<00:00, 37992.49it/s]


...


[INFO] Allocating reservations


100%|██████████| 4/4 [00:00<?, ?it/s]



[INFO] Adding preferences to meetings


100%|██████████| 6/6 [00:00<00:00, 3008.47it/s]

[INFO] Saving original solution
[INFO] Verifying solution
[INFO] Solution is correct
1000
36
16


In [7]:
monday = []
tuesday = []
wednesday = []
thursday = []
friday = []
saturday = []
for meeting in original_meetings:
    if meeting.day_name() == 'monday':
        monday.append(meeting)
    elif meeting.day_name() == 'tuesday':
        tuesday.append(meeting)
    elif meeting.day_name() == 'wednesday':
        wednesday.append(meeting)
    elif meeting.day_name() == 'thursday':
        thursday.append(meeting)
    elif meeting.day_name() == 'friday':
        friday.append(meeting)
    elif meeting.day_name() == 'saturday':
        saturday.append(meeting)
    else:
        raise Exception('Invalid day of week')

# Solving subparts using MIP
mip_solution = copy.deepcopy(original_solution)

In [8]:
## Monday
monday_cost, monday_allocations = mipPy({'meetings': monday, "classrooms": original_classrooms, "objectives": original_objectives}, instance)
for i in monday_allocations:
    if i['classroom_id'] != 0:
        allocate(mip_solution, i['meeting_id'], i['classroom_id'])

## Tuesday
tuesday_cost, tuesday_allocations = mipPy({'meetings': tuesday, "classrooms": original_classrooms, "objectives": original_objectives}, instance)
for i in tuesday_allocations:
    if i['classroom_id'] != 0:
        allocate(mip_solution, i['meeting_id'], i['classroom_id'])

## Wednesday
wednesday_cost, wednesday_allocations = mipPy({'meetings': wednesday, "classrooms": original_classrooms, "objectives": original_objectives}, instance)
for i in wednesday_allocations:
    if i['classroom_id'] != 0:
        allocate(mip_solution, i['meeting_id'], i['classroom_id'])

## Thursday
thursday_cost, thursday_allocations = mipPy({'meetings': thursday, "classrooms": original_classrooms, "objectives": original_objectives}, instance)
for i in thursday_allocations:
    if i['classroom_id'] != 0:
        allocate(mip_solution, i['meeting_id'], i['classroom_id'])

## Friday
friday_cost, friday_allocations = mipPy({'meetings': friday, "classrooms": original_classrooms, "objectives": original_objectives}, instance)
for i in friday_allocations:
    if i['classroom_id'] != 0:
        allocate(mip_solution, i['meeting_id'], i['classroom_id'])

## Saturday
saturday_cost, saturday_allocations = mipPy({'meetings': saturday, "classrooms": original_classrooms, "objectives": original_objectives}, instance)
for i in saturday_allocations:
    if i['classroom_id'] != 0:
        allocate(mip_solution, i['meeting_id'], i['classroom_id'])

## Verifying MIP solution
verifier(mip_solution)
mip_solution['objectives'].print()
total_cost = monday_cost + tuesday_cost + wednesday_cost + thursday_cost + friday_cost + saturday_cost
print(f'[INFO] Total cost: {total_cost}')
# exit()

mip_objectives = copy.deepcopy(mip_solution['objectives'])

[INFO] Starting MIP
[INFO] MIP finished
[INFO] Starting MIP
[INFO] MIP finished
[INFO] Starting MIP
[INFO] MIP finished
[INFO] Starting MIP
[INFO] MIP finished
[INFO] Starting MIP
[INFO] MIP finished
[INFO] Starting MIP
[INFO] MIP finished
[INFO] Verifying solution
[INFO] Solution is correct

====================== Objectives ======================
Idleness   :  0
Deallocated:  2615
Standing   :  282
[INFO] Total cost: 2897.0


In [42]:
filename = 'input-seed-6-size-1000.json'
input = filename
if filename != 'instance.json':
    filename = filename.split('.')[0].split('input-')[1]
else:
    filename = filename.split('.')[0]
filename = f'output-instance-{filename}'

current_directory = os.getcwd()
print('Diretório de Trabalho Atual:', current_directory)

lahc_multi_data = []
number_of_solutions_lahc = []
for i in range(5):
    with open(f'../json/output/lahc-multi/{filename}-params-seed-{i + 1}-time-900.json', 'r') as file:
        data = json.load(file)
        number_of_solutions_lahc.append(len(data[0]))
        data = [Objectives(i["idleness"], i["deallocated"], i["standing"]) for i in data[0]]
        lahc_multi_data = lahc_multi_data + data
print(len(lahc_multi_data))
lahc_multi_data = remove_objectives_duplicates(lahc_multi_data)
pareto_front = nondominated_sort(lahc_multi_data)[0]
print(len(lahc_multi_data))
lahc_multi_data = [lahc_multi_data[i] for i in pareto_front]
print(len(lahc_multi_data))

nsgaII_data = []
number_of_solutions_nsgaII = []
for i in range(5):
    with open(f'../json/output/nsgaII/{filename}-params-seed-{i + 1}-time-900.json', 'r') as file:
        data = json.load(file)
        number_of_solutions_nsgaII.append(len(data[0]))
        data = [Objectives(i["idleness"], i["deallocated"], i["standing"]) for i in data[0]]
        nsgaII_data = nsgaII_data + data
print(len(nsgaII_data))
nsgaII_data = remove_objectives_duplicates(nsgaII_data)
print(len(nsgaII_data))
pareto_front = nondominated_sort(nsgaII_data)[0]
nsgaII_data = [nsgaII_data[i] for i in pareto_front]
print(len(nsgaII_data))

mip_data = [mip_objectives]

print(f'[INFO] Number of solutions LAHC: {number_of_solutions_lahc}')
print(f'[INFO] Number of solutions NSGAII: {number_of_solutions_nsgaII}')
print(len(nondominated_sort(lahc_multi_data)))
print(len(nondominated_sort(nsgaII_data)))

Diretório de Trabalho Atual: d:\faculdade\monografia\mono_2\src
216
[INFO] Removing objectives duplicates
51
50
500
[INFO] Removing objectives duplicates
499
456
[INFO] Number of solutions LAHC: [60, 14, 38, 45, 59]
[INFO] Number of solutions NSGAII: [100, 100, 100, 100, 100]
1
1


In [23]:
x_axis_lahc_multi = [i.idleness for i in lahc_multi_data]
y_axis_lahc_multi = [i.deallocated for i in lahc_multi_data]
z_axis_lahc_multi = [i.standing for i in lahc_multi_data]

x_axis_nsgaII = [i.idleness for i in nsgaII_data]
y_axis_nsgaII = [i.deallocated for i in nsgaII_data]
z_axis_nsgaII = [i.standing for i in nsgaII_data]

x_axis_mip = [i.idleness for i in mip_data]
y_axis_mip = [i.deallocated for i in mip_data]
z_axis_mip = [i.standing for i in mip_data]

# x_axis_initial = [i.idleness for i in initial_population]
# y_axis_initial = [i.deallocated for i in initial_population]
# z_axis_initial = [i.standing for i in initial_population]

x_axis_lahc_multi, x_axis_nsgaII, x_axis_mip = normalize_values(x_axis_lahc_multi, x_axis_nsgaII, x_axis_mip)
y_axis_lahc_multi, y_axis_nsgaII, y_axis_mip = normalize_values(y_axis_lahc_multi, y_axis_nsgaII, y_axis_mip)
z_axis_lahc_multi, z_axis_nsgaII, z_axis_mip = normalize_values(z_axis_lahc_multi, z_axis_nsgaII, z_axis_mip)

In [29]:
hv_lahc_data = []
for i in range(len(x_axis_lahc_multi)):
    hv_lahc_data.append([x_axis_lahc_multi[i], y_axis_lahc_multi[i], z_axis_lahc_multi[i]])

hv_nsgaII_data = []
for i in range(len(x_axis_nsgaII)):
    hv_nsgaII_data.append([x_axis_nsgaII[i], y_axis_nsgaII[i], z_axis_nsgaII[i]])

[[0.9639415354963058, 0.09024211298606016, 0.9207207207207208], [0.06424670735624799, 0.8030080704328687, 0.013813813813813814], [0.06336331513009959, 0.8176815847395451, 0.013813813813813814], [0.06416639897205269, 0.8096111518708731, 0.013813813813813814], [0.06392547381946675, 0.8121790168745414, 0.013813813813813814], [0.06480886604561516, 0.7989728539985327, 0.013813813813813814], [0.0659331834243495, 0.7923697725605282, 0.013813813813813814], [0.06954706071313844, 0.7802641232575201, 0.013813813813813814], [0.06689688403469322, 0.7865003668378576, 0.013813813813813814], [0.957516864760681, 0.09574468085106383, 0.8996996996996997], [0.06890459363957598, 0.7971386647101981, 0.013813813813813814], [0.05878573723096691, 0.8294203961848863, 0.013813813813813814], [0.060231288146482494, 0.8114453411592076, 0.013813813813813814], [0.06416639897205269, 0.801540719002201, 0.013813813813813814], [0.06151622229360745, 0.8077769625825385, 0.013813813813813814], [0.06304208159331834, 0.802274

In [40]:
reference_point = np.array([2.0, 2.0, 2.0])

hv_calculator = Hypervolume(reference_point)
hypervolume = hv_calculator.do(np.array(hv_lahc_data))
print("Hypervolume LAHC:", hypervolume)

hypervolume = hv_calculator.do(np.array(hv_nsgaII_data))
print("Hypervolume NSGA-II:", hypervolume)

Hypervolume LAHC: 5.647212876930048
Hypervolume NSGA-II: 5.737214818872731


In [25]:
fig = go.Figure()

lahc_multi_scatter = go.Scatter3d(x=x_axis_lahc_multi, y=y_axis_lahc_multi, z=z_axis_lahc_multi, mode='markers', name='MOLA', hovertemplate='<b>Idleness</b>: %{x}'+'<br><b>Deallocated</b>: %{y}<br><b>Standing</b>: %{z}', marker=dict(color='red', size=2))
nsgaII_scatter = go.Scatter3d(x=x_axis_nsgaII, y=y_axis_nsgaII, z=z_axis_nsgaII, mode='markers', name='NSGA-II', hovertemplate='<b>Idleness</b>: %{x}'+'<br><b>Deallocated</b>: %{y}<br><b>Standing</b>: %{z}', marker=dict(color='blue', size=2))
mip_scatter = go.Scatter3d(x=x_axis_mip, y=y_axis_mip, z=z_axis_mip, mode='markers', name='MIP', hovertemplate='<b>Idleness</b>: %{x}'+'<br><b>Deallocated</b>: %{y}<br><b>Standing</b>: %{z}', marker=dict(color='green', size=2))
# initial_scatter = go.Scatter3d(x=x_axis_initial, y=y_axis_initial, z=z_axis_initial, mode='markers', name='Initial Population', hovertemplate='<b>Idleness</b>: %{x}'+'<br><b>Deallocated</b>: %{y}<br><b>Standing</b>: %{z}')

fig.add_trace(lahc_multi_scatter)
fig.add_trace(nsgaII_scatter)
fig.add_trace(mip_scatter)
# fig.add_trace(initial_scatter)

fig.update_layout(scene=dict(xaxis_title='Idleness', yaxis_title='Deallocated', zaxis_title='Standing'))

fig.update_layout(title=f'Solutions Comparison ({input})')

fig.update_layout(legend=dict(title='Algorithms'))

# fig = go.Figure(data=[
#         go.Scatter3d(x=x_axis_nsgaII, y=y_axis_nsgaII, z=z_axis_nsgaII, mode='markers', marker=dict(color='red', size=7)),
#         go.Scatter3d(x=x_axis_lahc_multi, y=y_axis_lahc_multi, z=z_axis_lahc_multi, mode='markers', marker=dict(color='blue', size=7)),
#         go.Scatter3d(x=x_axis_mip, y=y_axis_mip, z=z_axis_mip, mode='markers', marker=dict(color='green', size=7))
#     ]
# )

# fig.update_layout(scene=dict(xaxis_title='Idleness', yaxis_title='Deallocated', zaxis_title='Standing'))

# Exibir o gráfico
# fig.show()
fig.write_image(f"../graphics/tables/{input.split('.')[0]}.png", engine='kaleido')